In [1]:
import os
import asyncio
import yaml
from dotenv import load_dotenv

# Semantic Kernel imports
from semantic_kernel import Kernel
from semantic_kernel.agents import ChatCompletionAgent,AgentGroupChat
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion
from semantic_kernel.connectors.search_engine import GoogleConnector
from semantic_kernel.core_plugins import WebSearchEnginePlugin
from semantic_kernel.functions import KernelArguments
from semantic_kernel.connectors.ai.function_choice_behavior import FunctionChoiceBehavior

# Updated imports for chat history and message content
from semantic_kernel.contents.chat_history import ChatHistory
from semantic_kernel.contents.chat_message_content import ChatMessageContent
from semantic_kernel.contents.utils.author_role import AuthorRole
from semantic_kernel.functions.kernel_function_from_prompt import KernelFunctionFromPrompt

In [2]:
load_dotenv()

True

## Creating the Kernel

In [23]:

# Here we are adding  AzureChatCompletion service and a Google WebSearch plugin as a tool.
def create_kernel(service_id: str) -> Kernel:
    kernel = Kernel()
    kernel.add_service(AzureChatCompletion(service_id=service_id))
    
    # Get and configure the prompt execution settings for auto function calling.
    settings = kernel.get_prompt_execution_settings_from_service_id(service_id)
    settings.function_choice_behavior = FunctionChoiceBehavior.Auto()
    
    # Add the Google WebSearch plugin (if API keys are set)
    google_api_key = os.getenv("GOOGLE_API_KEY")
    google_search_engine_id = "44b41c05ad10f4559"  # TODO fget this from env variables
    if google_api_key and google_search_engine_id:
        connector = GoogleConnector(
            api_key=google_api_key,
            search_engine_id=google_search_engine_id
        )
        web_plugin = WebSearchEnginePlugin(connector)
        kernel.add_plugin(web_plugin, plugin_name="WebSearch")
    return kernel


# Loading the Instructions for the Agents fom the config files

In [3]:

# Loading the YAML configurations for agents and tasks TODO utilize plugins for this
def load_configurations():
    with open("../crewai/config/agents.yaml", "r") as f:
        agents_config = yaml.safe_load(f)
    with open("../crewai/config/tasks.yaml", "r") as f:
        tasks_config = yaml.safe_load(f)
    return agents_config, tasks_config

# Building full instructions for an agent from its config and the corresponding task.
def build_agent_instructions(agent_config: dict, task_config: dict) -> str:
    
    instructions = (
        f"Role: {agent_config.get('role','')}\n"
        f"Goal: {agent_config.get('goal','')}\n"
        f"Backstory: {agent_config.get('backstory','')}\n\n"
        f"Task:\n{task_config.get('description','')}\n\n"
        f"Expected Output:\n{task_config.get('expected_output','')}\n"
    )
    return instructions


In [25]:
# Helper to creatie a ChatHistory NOTE maybe we can use the groupchat hostory to achieve this
async def collect_response(agent: ChatCompletionAgent, input_text: str) -> str:

    history = ChatHistory(system_message=agent.instructions)
   
    history.add_message(ChatMessageContent(role=AuthorRole.USER, content=input_text))
    
    response_text = ""
    async for result in agent.invoke(history):
        response_text += result.content
    return response_text


## Create the Agents

In [26]:

#TODO handle the visual content generation
async def run_market_news_monitor(subject: str, agent_instr: str, kernel: Kernel) -> str:
    # Create the Market News Monitor agent
    agent = ChatCompletionAgent(
        service_id="market_news",  
        kernel=kernel,
        name="MarketNewsMonitor",
        instructions=agent_instr,
        arguments=KernelArguments()  
    )
    # Pass the subject as input.
    input_text = f"Subject: {subject}"
    response = await collect_response(agent, input_text)
    print("=== Market News Monitor Output ===")
    print(response)
    return response

async def run_data_analyst(subject: str, agent_instr: str, kernel: Kernel) -> str:
    # Create the Data Analyst agent
    agent = ChatCompletionAgent(
        service_id="data_analyst",
        kernel=kernel,
        name="DataAnalyst",
        instructions=agent_instr,
        arguments=KernelArguments()
    )
    input_text = f"Subject: {subject}"
    response = await collect_response(agent, input_text)
    print("=== Data Analyst Output ===")
    print(response)
    return response

async def run_content_creator(subject: str, combined_context: dict, agent_instr: str, kernel: Kernel) -> str:
    # Create the Content Creator agent
    agent = ChatCompletionAgent(
        service_id="content_creator",
        kernel=kernel,
        name="ContentCreator",
        instructions=agent_instr,
        arguments=KernelArguments()
    )
   
    input_text = (
        f"Subject: {subject}\n\n"
        f"Market News Analysis:\n{combined_context['market_news']}\n\n"
        f"Data Analysis:\n{combined_context['data_analyst']}\n\n"
        "Based on the above, create high-quality content."
    )
    response = await collect_response(agent, input_text)
    print("=== Content Creator Output ===")
    print(response)
    return response

async def run_quality_assurance(content: str, agent_instr: str, kernel: Kernel) -> str:
    # Create the Quality Assurance agent (Content Officer)
    agent = ChatCompletionAgent(
        service_id="quality_assurance",
        kernel=kernel,
        name="ContentOfficer",
        instructions=agent_instr,
        arguments=KernelArguments()
    )
    input_text = (
        "Please review and refine the following content ensuring it is formatted in markdown "
        "and structured as JSON with two keys: 'article' and 'social_media_posts'. "
        "The 'social_media_posts' value should be a list of objects, each with 'platform' and 'content'.\n\n"
        f"Content:\n{content}"
    )
    response = await collect_response(agent, input_text)
    print("=== Quality Assurance Output ===")
    print(response)
    return response


## Defining the Flow

In [27]:
async def flow(subject: str):
    # Load YAML configurations
    agents_config, tasks_config = load_configurations()

    # Subject for the content creation
    subject = subject

    # Build instructions for each agent by combining agent and task configs.
    market_news_instr = build_agent_instructions(
        agents_config["market_news_monitor_agent"],
        tasks_config["monitor_financial_news"]
    )
    data_analyst_instr = build_agent_instructions(
        agents_config["data_analyst_agent"],
        tasks_config["analyze_market_data"]
    )
    content_creator_instr = build_agent_instructions(
        agents_config["content_creator_agent"],
        tasks_config["create_content"]
    )
    quality_assurance_instr = build_agent_instructions(
        agents_config["quality_assurance_agent"],
        tasks_config["quality_assurance"]
    )

    # TODO see if single kernel works here instead creating seprate kernels
    kernel_market = create_kernel("market_news")
    kernel_data = create_kernel("data_analyst")
    kernel_content = create_kernel("content_creator")
    kernel_quality = create_kernel("quality_assurance")

    # Run the first two agents in parallel (Market News Monitor and Data Analyst)
    market_news_future = run_market_news_monitor(subject, market_news_instr, kernel_market)
    data_analyst_future = run_data_analyst(subject, data_analyst_instr, kernel_data)
    market_news_result, data_analyst_result = await asyncio.gather(market_news_future, data_analyst_future)

    # Combine the outputs from the first two agents
    combined_context = {
        "market_news": market_news_result,
        "data_analyst": data_analyst_result
    }

    # Run Content Creator agent with the combined context
    content_output = await run_content_creator(subject, combined_context, content_creator_instr, kernel_content)

    # Run Quality Assurance agent on the content output
    final_output = await run_quality_assurance(content_output, quality_assurance_instr, kernel_quality)

    # Print final structured output (should be a JSON-like string with 'article' and 'social_media_posts')
    print("\n=== FINAL STRUCTURED OUTPUT ===")
    print(final_output)
    return final_output


## Kiking off the flow

In [28]:

final_output = await flow("Inflation in the US and the impact on the stock market in 2024")

=== Data Analyst Output ===
**Comprehensive Analysis Report: Inflation in the US and Its Impact on the Stock Market in 2024**

**Executive Summary:**

The analysis focuses on understanding the intricate relationship between inflationary trends in the US and stock market dynamics in 2024. Leveraging advanced analytics tools, we processed large datasets to extract key insights on how inflation influences various market elements including investor sentiment and economic indicators. This report will guide content creation that resonates with investors seeking to navigate these complexities.

---

**Key Trends and Insights:**

1. **Inflation Trajectory and Causes:**
   - Based on recent data, US inflation in 2024 is projected to maintain moderate levels due to persistent supply chain improvements and energy market stabilization.
   - Inflationary pressures may also be influenced by robust consumer demand and government fiscal policies aimed at bolstering infrastructure and green initiatives

## Display the social media posts

In [29]:

import json, re, textwrap
from IPython.display import display, Markdown

def extract_json(text: str) -> str:
    """
    Extract JSON content from a string that may be wrapped in markdown code fences.
    """
    pattern = re.compile(r"```json\s*(\{.*\})\s*```", re.DOTALL)
    match = pattern.search(text)
    if match:
        return match.group(1)
    return text.strip()

def render_article_to_markdown(article: dict) -> str:
    """
    Converts the article dictionary (with keys like 'title', 'introduction', 'sections')
    into a Markdown-formatted string.
    """
    md = ""
    title = article.get("title", "")
    if title:
         md += f"# {title}\n\n"
    intro = article.get("introduction", "")
    if intro:
         md += f"{intro}\n\n"
    sections = article.get("sections", [])
    for section in sections:
         header = section.get("header", "")
         content = section.get("content", "")
         bullets = section.get("bullets", [])
         if header:
             md += f"## {header}\n\n"
         if content:
             md += f"{content}\n\n"
         if bullets:
             for bullet in bullets:
                  md += f"- {bullet}\n"
             md += "\n"
    return md

if final_output is None:
    print("Final output is None. Please check that your QA agent is producing an output.")
else:
    try:
        # Clean the output by extracting JSON if it's wrapped in code fences
        final_output_cleaned = extract_json(final_output)
        output_data = json.loads(final_output_cleaned)
        
        article = output_data.get("article", "")
        posts = output_data.get("social_media_posts", [])
        
        # If the article is a dict, convert it to Markdown text.
        if isinstance(article, dict):
            article_md = render_article_to_markdown(article)
        else:
            article_md = article
        
        # Display the blog post as Markdown
        display(Markdown(article_md))

    except Exception as e:
        print("Error parsing final output:", e)


**Blog Post: Navigating Inflation and Its Impact on the US Stock Market in 2024**

---

As 2024 unfolds, inflation in the United States takes center stage in financial discussions, with its tentacles reaching into every aspect of the stock market. Investors find themselves at the crossroads, steering their portfolios amidst inflationary currents that continue to redefine the economic landscape. This blog delves into the intricate dance of inflation trends and stock market dynamics, providing a comprehensive guide for investors primed to seize opportunities amid uncertainties.

### Understanding Inflation Trends and Their Market Manifestations

First, let's dissect the current inflation trajectory. By late 2023, inflation rates began to show signs of moderation after surging during the post-pandemic recovery period. But with the Consumer Price Index (CPI) and Producer Price Index (PPI) still hovering above the Federal Reserve’s target, investors await with bated breath for 2024's projected inflation stabilization around 3%-3.5%.

- **Data Visualization: Inflation Rate Trends (2019-2024)**
  - A line chart visualizing the fluctuation of inflation rates, highlighting key events that triggered spikes and declines, can help investors frame historical contexts with future expectations.

### Sectoral Dynamics: Winners and Losers in an Inflated Economy

Different sectors respond disparately to inflation. Therefore, tailoring investments to harness sectoral strengths could buffer portfolios against inflationary blows.

- **Technology and Growth Sectors** grapple with interest rate-induced volatility, affecting valuations and future cash flows. However, tech innovation continues to attract strategic investment.
- **Consumer Staples** often show resilience due to steady demand even as input costs climb.
- **Financials**, particularly banking, witness enhanced net interest margins, benefiting from interest rate increases accompanying inflation.
- **Energy and Commodities** stand poised to thrive on cost-driven price increases, although geopolitical tensions complicate stability.

- **Data Visualization: Sector Performance vs. Inflation Rate**
  - A comparative bar chart illustrating sector returns during inflation peaks will guide strategic allocation for both defensive and growth investors.

### Strategic Investment Movements: Harnessing Inflation

Understanding investor sentiment offers insight into prevailing market psychology. As inflation persists, investors may lean towards safe-haven assets, triggering a flight to gold and Treasury Inflation-Protected Securities (TIPS).

- **Interactive Tool: Inflation Impact Simulation**
  - Create a tool for users to input inflation rate predictions and visualize potential portfolio performance, boosting confidence in decision-making.

### Federal Reserve’s Tightrope: Balancing Policies and Market Sentiment

The Federal Reserve plays a pivotal role in moderating inflation through monetary policy adjustments. Anticipating its maneuvers could inform significant market movements.

- **Social Media Update: Fed Meeting Outcomes**

### Regulatory and Fiscal Considerations: Navigating the Policy Maze

The US government's fiscal measures—ranging from tax reforms to sectoral support—will also contribute to the broader inflation picture, with these steps potentially triggering market recalibrations.

### Action Steps for Investors

1. **Diversification Across Inflation-Resilient Sectors**: Balance portfolios by including staples, financials, and commodity investments.
2. **Hedge Against Inflation Risks**: Increase allocations to TIPS, commodities, and REITs.
3. **Continuous Monitoring of Economic Indicators**: Track CPI and PPI releases to gauge real-time inflation impacts on investments.

### Monthly Update: Keeping Pulse on Inflation

Introducing a monthly newsletter that delivers up-to-date insights on inflation trends, Fed policy impacts, and stock market shifts will empower investors to remain proactive and responsive.

---

Through these insights, our aim is to arm investors with the nuances of inflation's impact, providing strategic foresight into how to traverse 2024's economic terrain. By staying informed and agile, investors can better navigate the complexities of inflation, turning challenges into strategic opportunities. Engage with our dynamic content to stay ahead in the financial race, ensuring your investment strategies are well-equipped to weather any storm.

---

Stay updated with our [monthly inflation and market reports](#), interactive tools, and sector-specific guides. Be an informed investor and make smarter choices in today's evolving market.

## Display the Blog post

In [30]:
if final_output is None:
    print("Final output is None. Please check that your QA agent is producing an output.")
else:
    try:
        # Clean the output by extracting JSON if it's wrapped in code fences
        final_output_cleaned = extract_json(final_output)
        output_data = json.loads(final_output_cleaned)
        
        article = output_data.get("article", "")
        posts = output_data.get("social_media_posts", [])
        
        # If the article is a dict, convert it to Markdown text.
        if isinstance(article, dict):
            article_md = render_article_to_markdown(article)
        else:
            article_md = article
        
        # Display the blog post as Markdown
        display(Markdown(article_md))
        
    except Exception as e:
        print("Error parsing final output:", e)

**Blog Post: Navigating Inflation and Its Impact on the US Stock Market in 2024**

---

As 2024 unfolds, inflation in the United States takes center stage in financial discussions, with its tentacles reaching into every aspect of the stock market. Investors find themselves at the crossroads, steering their portfolios amidst inflationary currents that continue to redefine the economic landscape. This blog delves into the intricate dance of inflation trends and stock market dynamics, providing a comprehensive guide for investors primed to seize opportunities amid uncertainties.

### Understanding Inflation Trends and Their Market Manifestations

First, let's dissect the current inflation trajectory. By late 2023, inflation rates began to show signs of moderation after surging during the post-pandemic recovery period. But with the Consumer Price Index (CPI) and Producer Price Index (PPI) still hovering above the Federal Reserve’s target, investors await with bated breath for 2024's projected inflation stabilization around 3%-3.5%.

- **Data Visualization: Inflation Rate Trends (2019-2024)**
  - A line chart visualizing the fluctuation of inflation rates, highlighting key events that triggered spikes and declines, can help investors frame historical contexts with future expectations.

### Sectoral Dynamics: Winners and Losers in an Inflated Economy

Different sectors respond disparately to inflation. Therefore, tailoring investments to harness sectoral strengths could buffer portfolios against inflationary blows.

- **Technology and Growth Sectors** grapple with interest rate-induced volatility, affecting valuations and future cash flows. However, tech innovation continues to attract strategic investment.
- **Consumer Staples** often show resilience due to steady demand even as input costs climb.
- **Financials**, particularly banking, witness enhanced net interest margins, benefiting from interest rate increases accompanying inflation.
- **Energy and Commodities** stand poised to thrive on cost-driven price increases, although geopolitical tensions complicate stability.

- **Data Visualization: Sector Performance vs. Inflation Rate**
  - A comparative bar chart illustrating sector returns during inflation peaks will guide strategic allocation for both defensive and growth investors.

### Strategic Investment Movements: Harnessing Inflation

Understanding investor sentiment offers insight into prevailing market psychology. As inflation persists, investors may lean towards safe-haven assets, triggering a flight to gold and Treasury Inflation-Protected Securities (TIPS).

- **Interactive Tool: Inflation Impact Simulation**
  - Create a tool for users to input inflation rate predictions and visualize potential portfolio performance, boosting confidence in decision-making.

### Federal Reserve’s Tightrope: Balancing Policies and Market Sentiment

The Federal Reserve plays a pivotal role in moderating inflation through monetary policy adjustments. Anticipating its maneuvers could inform significant market movements.

- **Social Media Update: Fed Meeting Outcomes**

### Regulatory and Fiscal Considerations: Navigating the Policy Maze

The US government's fiscal measures—ranging from tax reforms to sectoral support—will also contribute to the broader inflation picture, with these steps potentially triggering market recalibrations.

### Action Steps for Investors

1. **Diversification Across Inflation-Resilient Sectors**: Balance portfolios by including staples, financials, and commodity investments.
2. **Hedge Against Inflation Risks**: Increase allocations to TIPS, commodities, and REITs.
3. **Continuous Monitoring of Economic Indicators**: Track CPI and PPI releases to gauge real-time inflation impacts on investments.

### Monthly Update: Keeping Pulse on Inflation

Introducing a monthly newsletter that delivers up-to-date insights on inflation trends, Fed policy impacts, and stock market shifts will empower investors to remain proactive and responsive.

---

Through these insights, our aim is to arm investors with the nuances of inflation's impact, providing strategic foresight into how to traverse 2024's economic terrain. By staying informed and agile, investors can better navigate the complexities of inflation, turning challenges into strategic opportunities. Engage with our dynamic content to stay ahead in the financial race, ensuring your investment strategies are well-equipped to weather any storm.

---

Stay updated with our [monthly inflation and market reports](#), interactive tools, and sector-specific guides. Be an informed investor and make smarter choices in today's evolving market.

# TESTING THE GROUPCHAT AND STRATEGY PATTERN

### This needs work as it is not currently working

In [6]:
from semantic_kernel.agents.strategies.selection.kernel_function_selection_strategy import (
    KernelFunctionSelectionStrategy,
)
from semantic_kernel.agents.strategies.termination.kernel_function_termination_strategy import (
    KernelFunctionTerminationStrategy,
)
from semantic_kernel.contents import ChatHistoryTruncationReducer

def _create_kernel_with_chat_completion(service_id: str) -> Kernel:
    kernel = Kernel()
    kernel.add_service(AzureChatCompletion(service_id=service_id))
    return kernel

agents_config, tasks_config = load_configurations()
market_news_instr = build_agent_instructions(
    agents_config["market_news_monitor_agent"],
    tasks_config["monitor_financial_news"]
)
data_analyst_instr = build_agent_instructions(
    agents_config["data_analyst_agent"],
    tasks_config["analyze_market_data"]
)
content_creator_instr = build_agent_instructions(
    agents_config["content_creator_agent"],
    tasks_config["create_content"]
)
quality_assurance_instr = build_agent_instructions(
    agents_config["quality_assurance_agent"],
    tasks_config["quality_assurance"]
)

# NOTE you get errors if you have spaces in the name 
MARKET_ANALYST_NAME = "Market_Analyst"
DATA_ANALYST_NAME = "Data_Analyst"
CONTENT_CREATOR_NAME = "Content_Creator"
CONTENT_OFFICER_NAME = "Content_Officer"

market_news_monitor_agent = ChatCompletionAgent(
    service_id=MARKET_ANALYST_NAME, 
    kernel=_create_kernel_with_chat_completion("market_analyst"),
    name=MARKET_ANALYST_NAME,
    instructions=market_news_instr,
    arguments=KernelArguments()  
    )

data_analyst_agent = ChatCompletionAgent(
        service_id=DATA_ANALYST_NAME,
        kernel=_create_kernel_with_chat_completion("data_analyst"),
        name=DATA_ANALYST_NAME,
        instructions=data_analyst_instr,
        arguments=KernelArguments()
    )
content_creator_agent = ChatCompletionAgent(
        service_id=CONTENT_CREATOR_NAME,
        kernel=_create_kernel_with_chat_completion("content_creator"),
        name=CONTENT_CREATOR_NAME,
        instructions=content_creator_instr,
      arguments=KernelArguments()
)

quality_assurance_agent = ChatCompletionAgent(
        service_id=CONTENT_OFFICER_NAME,
        kernel=_create_kernel_with_chat_completion("quality_assurance"),
        name=CONTENT_OFFICER_NAME,
        instructions=quality_assurance_instr,
        arguments=KernelArguments()
    )


selection_function = KernelFunctionFromPrompt(
    function_name="selection",
    prompt=f"""
    Determine which participant takes the next turn in a conversation based on the the most recent participant.
    State only the name of the participant to take the next turn.
    No participant should take more than one turn in a row.

    Choose only from these participants:
    - {MARKET_ANALYST_NAME}
    - {DATA_ANALYST_NAME}
    - {CONTENT_CREATOR_NAME}
    - {CONTENT_OFFICER_NAME}

    Always follow these rules when selecting the next participant:
    - After user input, it is {MARKET_ANALYST_NAME} it's turn.
    - After {MARKET_ANALYST_NAME} provides content, it is {DATA_ANALYST_NAME}'s turn.
    - After {MARKET_ANALYST_NAME} and {DATA_ANALYST_NAME} provide content, it is {CONTENT_CREATOR_NAME}'s turn to create the content.
    - After {CONTENT_CREATOR_NAME} creates content, it is {CONTENT_OFFICER_NAME}'s turn to vet and review the content.

    History:
    {{{{$history}}}}
    """,
)

TERMINATION_KEYWORD = "yes"

termination_function = KernelFunctionFromPrompt(
    function_name="termination",
    prompt=f"""
        Examine the RESPONSE and determine whether the content has been deemed satisfactory.
        If content is satisfactory, respond with a single word without explanation: {TERMINATION_KEYWORD}.
        If specific suggestions are being provided, it is not satisfactory.
        If no correction is suggested, it is satisfactory.

        RESPONSE:
        {{{{$history}}}}
        """,
)

history_reducer = ChatHistoryTruncationReducer(target_count=1)

chat = AgentGroupChat(
    agents=[market_news_monitor_agent, data_analyst_agent, content_creator_agent, quality_assurance_agent],
    selection_strategy=KernelFunctionSelectionStrategy(
        function=selection_function,
        kernel=_create_kernel_with_chat_completion("selection"),
        result_parser=lambda result: str(result.value[0]) if result.value is not None else MARKET_ANALYST_NAME or DATA_ANALYST_NAME,
        agent_variable_name="agents",
        history_variable_name="history",
    ),
        termination_strategy=KernelFunctionTerminationStrategy(
        agents=[quality_assurance_agent],
        function=termination_function,
        kernel=_create_kernel_with_chat_completion("termination"),
        result_parser=lambda result: TERMINATION_KEYWORD in str(result.value[0]).lower(),
        history_variable_name="history",
        maximum_iterations=10,
        history_reducer=history_reducer,
    ),
)

In [7]:
async for response in chat.invoke():
    print(f"# {response.role} - {response.name or '*'}: '{response.content}'")

# AuthorRole.ASSISTANT - Market_Analyst: 'Subject: Cryptocurrency Markets

---

**Summary Report: Cryptocurrency Market News and Trends**

**Date: [Current Date]**

**1. Bitcoin and Ethereum Price Dynamics**
- **Recent Trends:** Bitcoin (BTC) and Ethereum (ETH) have shown increased volatility recently. BTC is navigating around the $55,000 mark, while ETH hovers near $3,600. Both cryptocurrencies have exhibited large intraday swings, typical of the reactivity to macroeconomic cues.
- **Implications:** Price stability remains uncertain. Investors should prepare for speculative trading environments. Content should offer strategies on navigating volatility, possibly focusing on short-term technical outlooks.

**2. Regulatory Developments:**
- **U.S. Regulation:**
  - **Biden Administration's Executive Order**: The recently signed executive order aims to create a framework for regulating digital assets, emphasizing consumer protection and market integrity.
  - **Impact:** This signals poten